In [1]:
import torch
from torch.utils.data import Dataset

import torchaudio
import torchaudio.transforms

import torchvision


import sys, os

from pprint import pprint

from tqdm.autonotebook import tqdm

import json

import numpy as np

import matplotlib.pylab as plt
import seaborn as sns

import librosa
import librosa.display

import pandas as pd

from pathlib import Path

import gc

MANUAL_SEED = 69

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from datetime import date
from datetime import datetime

import os.path
from os import path
  
import json

import time

import copy

from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import KFold

import random

from pprint import pformat

import math
import pathlib

from torchaudio_augmentations import * 

import wandb


In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
def make_dir_if_absent(dir_path):
  
  if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    

In [4]:
class ComposeTransform:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, audio_data):
        for t in self.transforms:
            audio_data = t(audio_data)
        return audio_data
    
    def __repr__(self):
        repr_list = []

        for t in self.transforms:
            repr_list.append(t.__repr__())

        return str(repr_list)

In [5]:
class IdentityTransform:
  def __init__(self):
    pass

  def __call__(self, audio_data):
    return audio_data

  def __repr__(self):
    
    return str( { "transform_name": "IdentityTransform" } )

In [6]:
class StandardizeTransform:
  def __init__(self, mean, std):
    self.mean = mean
    self.std = std

  def __call__(self, audio_data):
    return (audio_data - self.mean) / self.std

  def __repr__(self):
    
    return str(
      {
        "transform_name": "StandardizeTransform",
        "mean": self.mean,
        "std": self.std 
      }
    )

In [7]:
class FMADataset(Dataset):

  def __init__(
    self, path, data_transforms_train, data_transforms_eval, data_type
  ):
    self.path = path
    self.data_type = data_type,
    self.stage = None
    self.data_transforms_train = data_transforms_train
    self.data_transforms_eval = data_transforms_eval
    self.data_paths = self._load_audio_list()

  def __len__(self):
    return len(self.data_paths)

  def __getitem__(self, idx):

    data = torch.load(self.data_paths[idx])

    if self.stage == "train" and self.data_transforms_train is not None:
      data = self.data_transforms_train(data)

    if (
      self.stage == "val" or self.stage == "test"
    ) and self.data_transforms_eval is not None:
      data = self.data_transforms_eval(data)

    label_one_hot = self._label_from_str_to_one_hot(
      self.data_paths[idx].split("/")[-2]
    )

    return data, label_one_hot

  
  def _label_from_str_to_one_hot(self, label_str: str): 
  
    if label_str == "Pop":
      return torch.tensor([1, 0, 0, 0, 0, 0]).float()
    
    if label_str == "Hip-Hop":
      return torch.tensor([0, 1, 0, 0, 0, 0]).float()
    
    if label_str == "Electronic":
      return torch.tensor([0, 0, 1, 0, 0, 0]).float()
    
    if label_str == "Rock":
      return torch.tensor([0, 0, 0, 1, 0, 0]).float()

    if label_str == "Folk":
      return torch.tensor([0, 0, 0, 0, 1, 0]).float()

    if label_str == "Jazz":
      return torch.tensor([0, 0, 0, 0, 0, 1]).float()
    
  
  def _load_audio_list(self):
    
    audio_path_list = []
    
    for path, subdirs, files in os.walk(self.path):
      for name in files:
          
        file_audio_path = os.path.join(path, name)
        
        audio_path_list.append(file_audio_path)

    return sorted(audio_path_list, reverse=True)
        
        

In [8]:
DATASET_SIZE = "s"
DATASET_TYPE = "mel_spectrogram"
DATASET_FOLDER = f"./data/{DATASET_TYPE}"

DATASET_NUM_SAMPLES_PER_SECOND = 8000
DATASET_NUM_CHANNELS = 1
N_FFT = 1024
WIN_LENGTH = None
HOP_LENGTH = 128
N_MELS = 128

DATASET_BASE_NAME = f"fma_{DATASET_SIZE}_resampled_{DATASET_NUM_SAMPLES_PER_SECOND}_rechanneled_{DATASET_NUM_CHANNELS}"
DATASET_NAME = f"{DATASET_BASE_NAME}_n_fft_{N_FFT}_win_length_{WIN_LENGTH}_hop_length_{HOP_LENGTH}_n_mels_{N_MELS}"


dataset_path = f"{DATASET_FOLDER}/{DATASET_NAME}"



SUMMARY_STATISTICS_PATH = f"./data/summary_statistics/{DATASET_BASE_NAME}/{DATASET_BASE_NAME}_summary_statistics.json"

In [9]:
summary_statistics_dict = json.load(open(SUMMARY_STATISTICS_PATH))

In [10]:
identity_transform = IdentityTransform()

standardize_transform = StandardizeTransform(
  mean=summary_statistics_dict[f"{DATASET_TYPE}_mean"],
  std=summary_statistics_dict[f"{DATASET_TYPE}_std"]
)

In [11]:
fma_data_transforms_train = ComposeTransform(
  [ 
    standardize_transform,
  ]
)

fma_data_transforms_eval = ComposeTransform(
  [ 
    standardize_transform,
  ] 
)

In [12]:
fma_dataset = FMADataset(
  path=dataset_path, 
  data_transforms_train=fma_data_transforms_train,
  data_transforms_eval=fma_data_transforms_eval,
  data_type=DATASET_TYPE
)

In [13]:
TRAIN_VAL_PERCENTAGE = 0.9

full_size = len(fma_dataset)
train_val_size = int(TRAIN_VAL_PERCENTAGE * full_size)
test_size = full_size - train_val_size

In [14]:
generator=torch.Generator().manual_seed(MANUAL_SEED)

fma_dataset_train_val, fma_dataset_test = torch.utils.data.random_split(
  fma_dataset, [train_val_size, test_size], generator
)

In [15]:
TRAIN_PERCENTAGE = 0.8

full_size = train_val_size
train_size = int(TRAIN_PERCENTAGE * full_size)
val_size = full_size - train_size

In [16]:
fma_dataset_train, fma_dataset_val = torch.utils.data.random_split(
  fma_dataset_train_val, [train_size, val_size], generator
)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
def count_num_trainable_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [20]:
def gen_train_id(): return datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

In [21]:
def save_dict_to_disk(dict, full_path):

  make_dir_if_absent("/".join(full_path.split("/")[:-1]))

  with open(full_path, 'w') as fp:
    json.dump(dict, fp)

def load_dict_from_disk(full_path):
  
  with open(full_path) as json_file:
    dict_from_disk = json.load(json_file)
  
  return dict_from_disk

In [22]:
def store_ckp(
  model, optimizer, ckp_dir, ckp_name, epoch, loss_train, loss_val, loss_test
):

  model_copy = copy.deepcopy(model)
  
  full_path_pickle = f"{ckp_dir}/{ckp_name}_epoch_{epoch}.pth"
  
  make_dir_if_absent(dir_path="/".join(full_path_pickle.split('/')[:-1]))
  
  torch.save(
    {
      'epoch': epoch,
      'model_state_dict': model_copy.cpu().state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss_train': loss_train,
      'loss_val': loss_val,
      'loss_test': loss_test,
    }, 
    full_path_pickle
  )
  
  torch.save(
    model_copy.cpu(), 
    full_path_pickle
  )

In [23]:
def load_ckp(ckp_path, perform_loading_sanity_check):

  loaded_model = torch.load(ckp_path)

  if perform_loading_sanity_check:

    loaded_model.eval()

    sanity_check_out = loaded_model(torch.rand((16, 1, 238000)))

  return loaded_model

In [24]:
def get_num_correct_preds(outputs, labels):
  
  output_pred_ind = torch.argmax(outputs, dim=1)
  labels_ind = torch.argmax(labels, dim=1)
  
  matching_mask = (output_pred_ind == labels_ind).float()
  
  num_correct_preds = matching_mask.sum()
  
  return num_correct_preds

In [25]:
def train_model(
  model, optimizer, criterion,
  batch_size, train_dl, val_dl, test_dl, 
  num_epochs, 
  device, 
  print_freq, ckp_freq, 
  ckp_dir, ckp_name,
  should_close_tqdm_prog_bars_when_done,
  limit_num_batches=None
):

  train_id = gen_train_id()
  
  training_logs = {
    "train_id": train_id,
    "accuracies": {},
    "losses": {}
  }
  
  model = model.to(device)

  wandb.init(
    project="GeNNus_CNN_mel_spec", entity="filetto-di-salmone"
  )
  wandb.watch(model)
  
  pbar_epochs = tqdm(range(num_epochs), colour="#9400d3")
  pbar_batches_train = tqdm(
    iter(train_dl), colour="#4169e1", leave=False,
  )
  pbar_batches_val = tqdm(
    iter(val_dl), colour="#008080", leave=False,
  )
  
  training_start_time = time.time()

  for epoch in range(num_epochs):

    running_loss_train = 0.0
    running_loss_val   = 0.0
    running_loss_test  = -1.0
    
    num_correct_preds_train = 0.0
    num_preds_train = 0.0
    accuracy_train = 0.0
    
    num_correct_preds_val = 0.0
    num_preds_val = 0.0
    accuracy_val = 0.0
    
    num_correct_preds_test = 0.0
    num_preds_test = 0.000000001
    accuracy_test = 0.0

    num_batches = 0
        
    ## BEGIN training step
    
    model.train()

    train_dl.dataset.dataset.stage="train"
    
    pbar_batches_train.reset()
    pbar_batches_val.reset()
    
    pbar_epochs.set_description(f"epoch {epoch}")
    pbar_batches_train.set_description(f"epoch {epoch}")
    pbar_batches_val.set_description  (f"epoch {epoch}")
    
    for batch_x, batch_y in iter(train_dl):

      if limit_num_batches is not None and num_batches <= limit_num_batches:

        inputs, labels = batch_x, batch_y
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        outputs = outputs.squeeze(-1)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        running_loss_train += loss.item() * batch_x.shape[0]
        
        num_correct_preds_train += get_num_correct_preds(outputs, labels)
        num_preds_train += outputs.shape[0]
      
      # else:
      #   print(f"skipped batch number: {num_batches}")

      num_batches += 1
      
      pbar_batches_train.update(1)
      
    
    ## END training step
    
    ## BEGIN validation step
    
    with torch.no_grad():
      
      model.eval()

      train_dl.dataset.dataset.stage="val"
      
      for batch_x, batch_y in iter(val_dl):

        inputs, labels = batch_x, batch_y
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        outputs = outputs.squeeze(-1)
        
        loss = criterion(outputs, labels)
        
        running_loss_val += loss.item() * batch_x.shape[0]
        
        num_correct_preds_val += get_num_correct_preds(outputs, labels)
        num_preds_val += outputs.shape[0]
        
        pbar_batches_val.update(1)
        
    ## END validation step
    
    ## BEGIN test step
    
    if (epoch + 1 == num_epochs):
      
      pbar_batches_test = tqdm(
        iter(test_dl), colour="#808000", leave=False,
      )
      pbar_batches_test.set_description  (f"epoch {epoch}")
    
      with torch.no_grad():
        
        model.eval()

        train_dl.dataset.dataset.stage="test"
        
        for batch_x, batch_y in iter(test_dl):

          inputs, labels = batch_x, batch_y
          inputs, labels = inputs.to(device), labels.to(device)
          
          outputs = model(inputs)
          outputs = outputs.squeeze(-1)
          
          loss = criterion(outputs, labels)
          
          running_loss_test += loss.item() * batch_x.shape[0]
          
          num_correct_preds_test += get_num_correct_preds(outputs, labels)
          num_preds_test += outputs.shape[0]
          
          pbar_batches_test.update(1)
        
    ## END test step
    
    accuracy_train = num_correct_preds_train / num_preds_train
    accuracy_val = num_correct_preds_val / num_preds_val
    accuracy_test = num_correct_preds_test / num_preds_test
    
    training_logs["accuracies"][str(epoch)] = {
      "accuracy_train": accuracy_train.cpu().item(),
      "accuracy_val": accuracy_val.cpu().item(),
    }
    training_logs["losses"][str(epoch)] = {
      "loss_train": running_loss_train,
      "loss_val": running_loss_val,
    }
    
    pbar_epochs.update(1)
    
    if ((epoch + 1) % print_freq == 0):  
      tqdm.write(
        f"epoch: {epoch + 1}\n" + 
        f"      train loss: {running_loss_train}, train acc: {accuracy_train}\n" + 
        f"      val loss  : {running_loss_val}, val acc  : {accuracy_val}\n"
      )
    
    if ((epoch + 1) == num_epochs):
      tqdm.write(
        f"      test loss : {running_loss_test}, test acc : {accuracy_test}"
      )
      
      training_logs["accuracies"][str(epoch)][
        "accuracy_test"
      ] = accuracy_test.cpu().item()
      
      training_logs["losses"][str(epoch)][
        "loss_test"
      ] = running_loss_test
      
    if (ckp_freq != None and (epoch + 1) % ckp_freq == 0):
      
      store_ckp(
        model=model, optimizer=optimizer, 
        ckp_dir=ckp_dir, ckp_name=ckp_name, epoch=epoch, 
        loss_train=running_loss_train, 
        loss_val=running_loss_val, 
        loss_test=running_loss_test
      )
  
  training_end_time = time.time()

  training_logs["training_time_secs"] = training_end_time - training_start_time

  if (should_close_tqdm_prog_bars_when_done):
    pbar_epochs.container.close()
    pbar_batches_train.close()
    pbar_batches_val.close()
    pbar_batches_test.close()
  
  return training_logs

## CNN

In [26]:
class CNN(nn.Module):
  def __init__(
    self, 
    num_layers, 
    kernel_sizes, strides, 
    in_channels, num_filters,
    pool_sizes, pool_strides,
    fc_in_features,
    use_batchnorm,
    dropout_p_conv=None, dropout_p_linear=None,
  ):
    super().__init__()

    self.num_layers = num_layers 
    self.kernel_sizes = kernel_sizes 
    self.strides = strides 
    self.in_channels = in_channels 
    self.num_filters = num_filters
    self.pool_sizes = pool_sizes 
    self.pool_strides = pool_strides

    self.use_batchnorm = use_batchnorm
    
    self.dropout_p_conv = dropout_p_conv
    self.dropout_p_linear = dropout_p_linear

    self.fc_in_features = fc_in_features

    self.bns = {
      "1": nn.BatchNorm2d(num_features=1),
      "2": nn.BatchNorm2d(num_features=2),
      "4": nn.BatchNorm2d(num_features=4),
      "6": nn.BatchNorm2d(num_features=6),
      "8": nn.BatchNorm2d(num_features=8),
      "16": nn.BatchNorm2d(num_features=16),
      "32": nn.BatchNorm2d(num_features=32),
      "64": nn.BatchNorm2d(num_features=64),
      "128": nn.BatchNorm2d(num_features=128),
      "256": nn.BatchNorm2d(num_features=256),
      "512": nn.BatchNorm2d(num_features=512)
    }
    
    self.convs = nn.Sequential()
    self.fc = nn.Sequential()
    
    for i in range(num_layers):
      
      conv_layer = nn.Conv2d(
        kernel_size=self.kernel_sizes[i],
        stride=self.strides[i],
        in_channels=in_channels,
        out_channels=self.num_filters[i]
      )
      torch.nn.init.xavier_uniform_(conv_layer.weight)

      
      pooling_layer = nn.MaxPool2d(
        kernel_size=self.pool_sizes[i],
        stride=self.pool_strides[i],
      )
      
      in_channels = self.num_filters[i]
      
      self.convs.add_module(name=f"conv_{i}", module=conv_layer)
      
      self.convs.add_module(name=f"pool_{i}", module=pooling_layer)
      
      if (self.use_batchnorm):
        self.convs.add_module(
          name=f"batchnorm_{i}", module=self.bns[str(self.num_filters[i])]
        )

      if (self.dropout_p_conv is not None):
        self.convs.add_module(
          name=f"dropout_{i}", module=nn.Dropout2d(p=self.dropout_p_conv)
        )
      
      self.convs.add_module(name=f"activ_{i}", module=nn.ReLU())

    if self.dropout_p_linear is not None:
      self.fc.add_module(
        name=f"dropout_before_fc_1", module=nn.Dropout(p=self.dropout_p_linear)
      )

    self.fc.add_module(
      name=f"fc_1", module=nn.Linear(
        in_features=self.fc_in_features, out_features=6
      )
    )

    self.fc.add_module(
      name="fc_1_activ", module=nn.Sigmoid()
    )


    
  
  def forward(self, x):    
    x = self.convs(x)

    x = torch.flatten(x, start_dim=1)

    x = self.fc(x)

    return x

  def get_model_setup(self):
    
    return {
      "num_layers": self.num_layers, 
      "kernel_sizes": self.kernel_sizes, 
      "strides": self.strides, 
      "in_channels": self.in_channels, 
      "num_filters": self.num_filters,
      "pool_sizes": self.pool_sizes, 
      "pool_strides": self.pool_strides,
      "dropout_p_conv": self.dropout_p_conv,
      "dropout_p_linear": self.dropout_p_linear,
    }

In [27]:
def plot_loss_curves(stats):
  epochs = stats["training_logs"]["losses"].keys()
  
  loss_train = [
    j["loss_train"] for j in stats["training_logs"]["losses"].values()
  ]
  
  loss_val = [j["loss_val"] for j in stats["training_logs"]["losses"].values()]

  sns.lineplot(x=epochs, y=loss_train, legend="full", label="train loss")

  sns.lineplot(x=epochs, y=loss_val, legend="full", label="val loss")

## K-fold cross validation for hyperparameter search

In [28]:
k_fold_cv_kernel_sizes =[ 2,  2,  2] #,  2]
k_fold_cv_pool_sizes   =[ 2,  2,  2] #,  2]
k_fold_cv_strides      =[ 2,  2,  1] #,  1]
k_fold_cv_pool_strides =[ 2,  2,  1] #,  1]
k_fold_cv_num_filters  =[ 1,  2,  4] #,  8]
# k_fold_cv_num_filters  =[ 2,  4,  8,  16]
# k_fold_cv_num_filters  =[ 4,  8,  16,  32]
k_fold_cv_fc_in_features = k_fold_cv_num_filters[-1] * 6 * 114

k_fold_cv_num_layers = len(k_fold_cv_kernel_sizes)

k_fold_cv_in_channels   = 1 # we always use mono audio in both cases!

K_FOLD_CV_DROPOUT_P_CONV = None
K_FOLD_CV_DROPOUT_P_LINEAR = 0.5
K_FOLD_CV_USE_BATCH_NORM = True

In [29]:
def cnn_factory(
  num_layers,
  kernel_sizes,
  strides,
  in_channels,
  num_filters,
  pool_sizes,
  pool_strides,
  dropout_p_conv, 
  dropout_p_linear,
  fc_in_features,
  use_batchnorm
):
  return CNN(
  num_layers=num_layers,
  kernel_sizes=kernel_sizes, 
  strides=strides, 
  in_channels=in_channels, 
  num_filters=num_filters,
  pool_sizes=pool_sizes,
  pool_strides=pool_strides,
  dropout_p_conv=dropout_p_conv,
  dropout_p_linear=dropout_p_linear,
  fc_in_features=fc_in_features,
  use_batchnorm=use_batchnorm
)

In [30]:
LR = 0.01
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-6
OPTIMIZER_NAME = "SGD"

In [31]:
def optimizer_factory(optimizer_name, model, lr, momentum, weight_decay):

  if optimizer_name == "SGD":
    optimizer = optim.SGD(
      model.parameters(), lr=lr, momentum=momentum, nesterov=True,
      weight_decay=weight_decay
    )

    optimizer_config = {
    "lr": lr, 
    "momentum": momentum, 
    "weight_decay": weight_decay,
    "nesterov": True
  }  

  elif optimizer_name == "Adam":

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    optimizer_config = {
    "lr": lr, 
    "weight_decay": weight_decay
  }  
    

  return optimizer, optimizer_config


In [32]:
K_FOLD_CV_NUM_FOLDS = 5
K_FOLD_CV_LIMIT_NUM_FOLDS = K_FOLD_CV_NUM_FOLDS

K_FOLD_CV_BATCH_SIZE = 32
K_FOLD_CV_LIMIT_NUM_BATCHES_PERCENTAGE = 0.1

K_FOLD_CV_NUM_EPOCHS = 120

K_FOLD_CV_PRINT_FREQ = int(K_FOLD_CV_NUM_EPOCHS * 0.1)
# K_FOLD_CV_PRINT_FREQ = 1000
# K_FOLD_CV_PRINT_FREQ = K_FOLD_CV_NUM_EPOCHS

K_FOLD_CV_CKP_FREQ = int(K_FOLD_CV_NUM_EPOCHS * 0.1)
# K_FOLD_CV_CKP_FREQ = 1000
# K_FOLD_CV_CKP_FREQ = K_FOLD_CV_NUM_EPOCHS

K_FOLD_CV_LOGS_FOLDER = f"./k_fold_cv/cnn/{DATASET_TYPE}"

K_FOLD_CV_CKP_FOLDER = K_FOLD_CV_LOGS_FOLDER

K_FOLD_CV_SHOULD_CLOSE_TQDM_PROG_BARS_WHEN_DONE=True

In [33]:
cv_models = [

  cnn_factory(
    num_layers=k_fold_cv_num_layers,
    kernel_sizes=k_fold_cv_kernel_sizes, 
    strides=k_fold_cv_strides, 
    in_channels=k_fold_cv_in_channels, 
    num_filters=k_fold_cv_num_filters,
    pool_sizes=k_fold_cv_pool_sizes,
    pool_strides=k_fold_cv_pool_strides,
    dropout_p_conv=K_FOLD_CV_DROPOUT_P_CONV,
    dropout_p_linear=K_FOLD_CV_DROPOUT_P_LINEAR,
    fc_in_features=k_fold_cv_fc_in_features,
    use_batchnorm=K_FOLD_CV_USE_BATCH_NORM
  ) for _ in range(0, K_FOLD_CV_NUM_FOLDS)

]

cv_criterions = [nn.CrossEntropyLoss() for _ in range(0, K_FOLD_CV_NUM_FOLDS)]

cv_opts = [
  optimizer_factory(
    optimizer_name=OPTIMIZER_NAME,
    model=cv_models[i],
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
  ) for i in range(0, K_FOLD_CV_NUM_FOLDS)
]

cv_optimizers = [opt for opt, _ in cv_opts]
cv_optimizers_configs = [opt_conf for _, opt_conf in cv_opts]

cv_train_dls = []
cv_val_dls = []
cv_test_dls = []

In [34]:
k_fold = KFold(n_splits=K_FOLD_CV_NUM_FOLDS, shuffle=True)

cv_train_idxs = []

for fold, (train_idxs, val_idxs) in enumerate(k_fold.split(fma_dataset_train_val)):

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_idxs)
  val_subsampler = torch.utils.data.SubsetRandomSampler(val_idxs)

  cv_train_dls.append(
    torch.utils.data.DataLoader(
      fma_dataset_train_val, batch_size=K_FOLD_CV_BATCH_SIZE, sampler=train_subsampler
    )
  )
  cv_train_idxs.append(train_idxs)

  cv_val_dls.append(
    torch.utils.data.DataLoader(
      fma_dataset_train_val, batch_size=K_FOLD_CV_BATCH_SIZE, sampler=val_subsampler
    )
  )

  cv_test_dls.append(
    torch.utils.data.DataLoader(
      fma_dataset_test, batch_size=K_FOLD_CV_BATCH_SIZE
    )
  )

k_fold_cv_limit_num_batches = int(
  len(list(cv_train_dls[0])) * K_FOLD_CV_LIMIT_NUM_BATCHES_PERCENTAGE
) + 1

cv_data_logs = {
  "data_type": DATASET_TYPE,
  "dataset_size": DATASET_SIZE,
  "batch_size": K_FOLD_CV_BATCH_SIZE,
  "num_samples_per_second": DATASET_NUM_SAMPLES_PER_SECOND,
  "num_channels": DATASET_NUM_CHANNELS,
  "train_transforms": fma_data_transforms_train.__repr__()
}

In [35]:
def perform_k_fold_cv(
  cv_id,
  cv_num_folds,
  cv_models, cv_optimizers, cv_criterions,
  batch_size, 
  cv_train_dls, cv_train_idxs, cv_val_dls, cv_test_dls, 
  cv_num_epochs, 
  cv_device, 
  cv_print_freq, cv_ckp_freq, 
  cv_ckp_dir,
  cv_should_close_tqdm_prog_bars_when_done,
  cv_limit_num_folds,
  cv_limit_num_batches=None
):

  cv_training_logs = {}

  pbar_folds = tqdm(range(cv_num_folds), colour="#b22222")

  for fold in pbar_folds:

    pbar_folds.set_description(f"fold {fold}")

    cv_ckp_fold_dir = f"{cv_ckp_dir}/fold_{fold}"

    if fold < cv_limit_num_folds:

      training_log = train_model(
        model=cv_models[fold], 
        optimizer=cv_optimizers[fold], criterion=cv_criterions[fold],
        batch_size=batch_size,
        train_dl=cv_train_dls[fold], val_dl=cv_val_dls[fold], 
        test_dl=cv_test_dls[fold],
        num_epochs=cv_num_epochs, 
        device=cv_device,
        print_freq=cv_print_freq, ckp_freq=cv_ckp_freq, 
        ckp_dir=cv_ckp_fold_dir, ckp_name=f"{cv_id}_fold_{fold}",
        should_close_tqdm_prog_bars_when_done=cv_should_close_tqdm_prog_bars_when_done,
        limit_num_batches=cv_limit_num_batches
      )

    cv_training_logs[str(fold)] = training_log

    pbar_folds.update(1)

  return cv_training_logs
  
  

In [36]:
def print_k_fold_cv_curves(
  k_fold_cv_stats, curve_name_singular, curve_name_plural, plot_title, 
  limit_to_n_folds=None
):
  
  num_folds = len(list(k_fold_cv_stats["training_logs"].keys()))
  num_folds_og = num_folds
  
  if limit_to_n_folds is not None and limit_to_n_folds <= num_folds:
    num_folds = limit_to_n_folds
  

  for fold in range(0, num_folds):

    fold_str = str(fold)

    losses_dict = k_fold_cv_stats["training_logs"][fold_str][curve_name_plural]

    epochs = losses_dict.keys()
    
    curve_train = [ j[f"{curve_name_singular}_train"] for j in losses_dict.values() ]

    curve_val = [ j[f"{curve_name_singular}_val"] for j in losses_dict.values() ]

    fig, ax = plt.subplots()

    sns.lineplot(
      x=epochs, y=curve_train, legend="full", 
      label=f"train {curve_name_singular}",  ax=ax
    )

    sns.lineplot(
      x=epochs, y=curve_val, legend="full", 
      label=f"val {curve_name_singular}",  ax=ax
    )

    fig.suptitle(plot_title)
    ax.set_title(f"{num_folds_og}-fold CV, fold {fold + 1}")
    
    epochs_as_list = list(epochs)
    PERCENTAGE_X_AXIS_TICKS = 0.2
    epoch_axis_display = epochs_as_list[
      ::int(len(epochs_as_list) * PERCENTAGE_X_AXIS_TICKS)
    ]
    ax.set_xticklabels(epoch_axis_display)
    ax.set_xticks(epoch_axis_display)

In [37]:
def print_avg_fold_cv_curves(
  k_fold_cv_stats, curve_name_singular, curve_name_plural, plot_title
):
  
  num_folds = len(list(k_fold_cv_stats["training_logs"].keys()))

  curves_train = []
  curves_val = []

  for fold in range(0, num_folds):

    fold_str = str(fold)

    losses_dict = k_fold_cv_stats["training_logs"][fold_str][curve_name_plural]

    epochs = losses_dict.keys()
    
    curves_train.append(
      [ j[f"{curve_name_singular}_train"] for j in losses_dict.values() ]
    )

    curves_val.append(
      [ j[f"{curve_name_singular}_val"] for j in losses_dict.values() ]
    )
  
  curves_train_np = np.asarray(curves_train)
  curves_val_np = np.asarray(curves_val)

  curve_train = np.average(curves_train_np, axis=0)
  curve_val = np.average(curves_val_np, axis=0)

  fig, ax = plt.subplots()

  sns.lineplot(
    x=epochs,y=curve_train,legend="full",
    label=f"train {curve_name_singular}", ax=ax
  )

  sns.lineplot(
    x=epochs,y=curve_val,legend="full",
    label=f"val {curve_name_singular}", ax=ax
  )

  fig.suptitle(plot_title)
  
  epochs_as_list = list(epochs)
  PERCENTAGE_X_AXIS_TICKS = 0.2
  epoch_axis_display = epochs_as_list[
    ::int(len(epochs_as_list) * PERCENTAGE_X_AXIS_TICKS)
  ]
  ax.set_xticklabels(epoch_axis_display)
  ax.set_xticks(epoch_axis_display)

  fig, ax =plt.subplots(2, 1)
  sns.lineplot(
    x=epochs,y=curve_train,legend="full",
    label=f"train {curve_name_singular}", ax=ax[0]
  )
  sns.lineplot(
    x=epochs,y=curve_val,legend="full",
    label=f"val {curve_name_singular}", ax=ax[1], color="orange"
  )

  ax[0].set_xticklabels(epoch_axis_display)
  ax[1].set_xticklabels(epoch_axis_display)
  ax[0].set_xticks(epoch_axis_display)
  ax[1].set_xticks(epoch_axis_display)
  fig.show()


In [38]:
K_FOLD_CV_RUN = True

K_FOLD_CV_PRINT_LOSS_CURVES_ACROSS_FOLDS = False
K_FOLD_CV_PRINT_ACC_CURVES_ACROSS_FOLDS  = False

K_FOLD_CV_PRINT_LOSS_CURVES_AVG_FOLDS = True
K_FOLD_CV_PRINT_ACC_CURVES_AVG_FOLDS  = True

K_FOLD_CV_PLOTS_LIMIT_TO_N_FOLDS = 1

In [ ]:
if K_FOLD_CV_RUN:

  k_fold_cv_id = gen_train_id()

  print(f"K-fold cross-validation ID: {k_fold_cv_id}")

  print(
    f"Total number of trainable parameters: \
    {count_num_trainable_parameters(cv_models[0])}"
  )

  k_fold_cv_training_logs = perform_k_fold_cv(
    cv_id=k_fold_cv_id,
    cv_num_folds=K_FOLD_CV_NUM_FOLDS,
    cv_models=cv_models, cv_optimizers=cv_optimizers, cv_criterions=cv_criterions,
    batch_size=K_FOLD_CV_BATCH_SIZE, 
    cv_train_dls=cv_train_dls, cv_train_idxs=cv_train_idxs, 
    cv_val_dls=cv_val_dls, cv_test_dls=cv_test_dls,
    cv_num_epochs=K_FOLD_CV_NUM_EPOCHS, 
    cv_device=device, 
    cv_print_freq=K_FOLD_CV_PRINT_FREQ, cv_ckp_freq=K_FOLD_CV_CKP_FREQ, 
    cv_ckp_dir=f"{K_FOLD_CV_CKP_FOLDER}/{k_fold_cv_id}",
    cv_should_close_tqdm_prog_bars_when_done=K_FOLD_CV_SHOULD_CLOSE_TQDM_PROG_BARS_WHEN_DONE,
    cv_limit_num_batches=k_fold_cv_limit_num_batches,
    cv_limit_num_folds=K_FOLD_CV_LIMIT_NUM_FOLDS
  )

  k_fold_cv_stats = {
    "k_fold_cv_id": k_fold_cv_id,
    "stats_type": "k_fold_cross_validation",
    "k_folds_cv_num_folds": K_FOLD_CV_NUM_FOLDS,
    "data_logs": cv_data_logs,
    "optimizer_config": cv_optimizers_configs[0], # all the same, one is enough
    "model_setup": cv_models[0].get_model_setup(), # all the same, one is enough
    "training_logs": k_fold_cv_training_logs,
  }

  save_dict_to_disk(
    dict=k_fold_cv_stats,
    full_path=f"{K_FOLD_CV_LOGS_FOLDER}/{k_fold_cv_id}/{k_fold_cv_id}_stats.json"
  )

  if K_FOLD_CV_PRINT_LOSS_CURVES_ACROSS_FOLDS:
    print_k_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats, 
      curve_name_plural="losses", curve_name_singular="loss",
      plot_title=f"Loss curves", 
      limit_to_n_folds=K_FOLD_CV_PLOTS_LIMIT_TO_N_FOLDS
    )
  
  if K_FOLD_CV_PRINT_ACC_CURVES_ACROSS_FOLDS:
    print_k_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats, 
      curve_name_plural="accuracies", curve_name_singular="accuracy",
      plot_title=f"Accuracy curves", 
      limit_to_n_folds=K_FOLD_CV_PLOTS_LIMIT_TO_N_FOLDS
    )
  if K_FOLD_CV_PRINT_LOSS_CURVES_AVG_FOLDS:
    print_avg_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats, 
      curve_name_singular="loss", curve_name_plural="losses", 
      plot_title=f"Loss curves\naverage across {K_FOLD_CV_NUM_FOLDS} folds"
    )
  if K_FOLD_CV_PRINT_ACC_CURVES_AVG_FOLDS:  
    print_avg_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats, 
      curve_name_singular="accuracy", curve_name_plural="accuracies", 
      plot_title=f"Accuracy curves\naverage across {K_FOLD_CV_NUM_FOLDS} folds"
    )

In [ ]:
if  K_FOLD_CV_RUN:

  k_fold_cv_id = "2022_12_11_14_47_50"
  
  k_fold_cv_stats_from_disk = load_dict_from_disk(
    full_path=f"{K_FOLD_CV_LOGS_FOLDER}/{k_fold_cv_id}/{k_fold_cv_id}_stats.json"
  )

  k_fold_cv_num_folds = k_fold_cv_stats_from_disk["k_folds_cv_num_folds"]

  if K_FOLD_CV_PRINT_LOSS_CURVES_ACROSS_FOLDS:
    print_k_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats_from_disk, 
      curve_name_plural="losses", curve_name_singular="loss",
      plot_title=f"Loss curves", 
      limit_to_n_folds=K_FOLD_CV_PLOTS_LIMIT_TO_N_FOLDS
    )
  
  if K_FOLD_CV_PRINT_ACC_CURVES_ACROSS_FOLDS:
    print_k_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats_from_disk, 
      curve_name_plural="accuracies", curve_name_singular="accuracy",
      plot_title=f"Accuracy curves", 
      limit_to_n_folds=K_FOLD_CV_PLOTS_LIMIT_TO_N_FOLDS
    )
  if K_FOLD_CV_PRINT_LOSS_CURVES_AVG_FOLDS:
    print_avg_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats_from_disk, 
      curve_name_singular="loss", curve_name_plural="losses", 
      plot_title=f"Loss curves\naverage across {k_fold_cv_num_folds} folds"
    )
  if K_FOLD_CV_PRINT_ACC_CURVES_AVG_FOLDS:  
    print_avg_fold_cv_curves(
      k_fold_cv_stats=k_fold_cv_stats_from_disk, 
      curve_name_singular="accuracy", curve_name_plural="accuracies", 
      plot_title=f"Accuracy curves\naverage across {k_fold_cv_num_folds} folds"
    )